<a href="https://colab.research.google.com/github/arafat04/bn-hi-MT-improvement-using-llm/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prompt formatting utilities

Simple helper functions to format prompts later on:

In [ ]:
template = """{source_lang}: {source_text}
{target_lang}: {target_text}"""

def apply_prompt(training=False, eos_token=None, **kwargs):
    # note: we strip because of potential trailing whitespace
    # we also provide a default value for target_text so that it can be omitted
    return template.format(**{"target_text": "", **kwargs}).strip() + ("" if not training or eos_token is None else eos_token)

def apply_prompt_n_shot(examples, n: int, eos_token: str, **kwargs):
    return (eos_token + "\n\n").join(
        [apply_prompt(**{"target_text": "", **example}) for example in examples[:n]] + [apply_prompt(**kwargs)]
    )

EXAMPLE_SENTENCES = [
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "খবরটা শুনে খুব খারাপ লাগলো।",
        "target_text": "खबर सुनकर बहुत दुख हुआ.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "এটার দাম কত?",
        "target_text": "इसकी कीमत कितनी होती है?",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "ঢাকা বাংলাদেশের রাজধানী।",
        "target_text": "ढाका बांग्लादेश की राजधानी है.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "রাস্তার দিকে মনোযোগ দিন।",
        "target_text": "सड़क पर ध्यान दें.",
    },
    {
        "source_lang": "Bengali",
        "target_lang": "Hindi",
        "source_text": "আমার মাথা ব্যথা করছে",
        "target_text": "मुझे सिर दर्द है.",
    }
]

In [ ]:
print(apply_prompt(source_lang="Bengali", source_text="হ্যালো.", target_lang="Hindi"))

Bengali: হ্যালো.
Hindi:


In [ ]:
print(apply_prompt(source_lang="Bengali", source_text="হ্যালো.", target_lang="Hindi", target_text="नमस्ते.", eos_token="</s>", training=True))

Bengali: হ্যালো.
Hindi: नमस्ते.</s>


In [ ]:
print(apply_prompt_n_shot(EXAMPLE_SENTENCES, 5, "</s>", source_lang="Bengali", source_text="হ্যালো.", target_lang="Hindi"))

Bengali: খবরটা শুনে খুব খারাপ লাগলো।
Hindi: खबर सुनकर बहुत दुख हुआ.</s>

Bengali: এটার দাম কত?
Hindi: इसकी कीमत कितनी होती है?</s>

Bengali: ঢাকা বাংলাদেশের রাজধানী।
Hindi: ढाका बांग्लादेश की राजधानी है.</s>

Bengali: রাস্তার দিকে মনোযোগ দিন।
Hindi: सड़क पर ध्यान दें.</s>

Bengali: আমার মাথা ব্যথা করছে
Hindi: मुझे सिर दर्द है.</s>

Bengali: হ্যালো.
Hindi:


In [ ]:

print(apply_prompt_n_shot(EXAMPLE_SENTENCES, 5, "</s>", source_lang="Bengali", source_text="হ্যালো.", target_lang="Hindi", target_text="नमस्ते."))

Bengali: খবরটা শুনে খুব খারাপ লাগলো।
Hindi: खबर सुनकर बहुत दुख हुआ.</s>

Bengali: এটার দাম কত?
Hindi: इसकी कीमत कितनी होती है?</s>

Bengali: ঢাকা বাংলাদেশের রাজধানী।
Hindi: ढाका बांग्लादेश की राजधानी है.</s>

Bengali: রাস্তার দিকে মনোযোগ দিন।
Hindi: सड़क पर ध्यान दें.</s>

Bengali: আমার মাথা ব্যথা করছে
Hindi: मुझे सिर दर्द है.</s>

Bengali: হ্যালো.
Hindi: नमस्ते.


In [ ]:
import torch
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel

/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/unsloth/__init__.py:22: UserWarning: Unsloth: 'CUDA_VISIBLE_DEVICES' is currently GPU-044e48b9-e794-cbfc-316e-1472ca268b72 but we require 'CUDA_VISIBLE_DEVICES=0'
We shall set it ourselves.
  warnings.warn(


## Load the base model

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-bnb-4bit",
    max_seq_length=4096,
    load_in_4bit=True,
)

/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: NVIDIA RTX A4000. Max memory: 15.724 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


In [ ]:
FastLanguageModel.for_inference(model)
sentence = "প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।"
prompt = apply_prompt(source_lang="Bengali", source_text=sentence, target_lang="Hindi")
print(prompt)
inputs = tokenizer(
    prompt,
    return_tensors="pt",
).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
tokenizer.batch_decode(outputs)

Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।
Hindi:


['<s> Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।\nHindi: प्रशिक्षण अलगोरिडम, परामिति, न्यायता सीमाबद्धता वा अन्यान्य प्रयोग पद्धति एवं वैशिष्ट्य संपर्के तथ्य।\n\n## Definition of Algorithm\n\nAn algorithm is a finite set of well-defined instructions for solving a problem.\n']

In [ ]:
prompt = apply_prompt_n_shot(EXAMPLE_SENTENCES, 5, eos_token=tokenizer.eos_token, source_lang="Bengali", source_text=sentence, target_lang="Hindi")
print(prompt)
inputs = tokenizer(
    prompt,
    return_tensors="pt",
).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
tokenizer.batch_decode(outputs)

Bengali: খবরটা শুনে খুব খারাপ লাগলো।
Hindi: खबर सुनकर बहुत दुख हुआ.</s>

Bengali: এটার দাম কত?
Hindi: इसकी कीमत कितनी होती है?</s>

Bengali: ঢাকা বাংলাদেশের রাজধানী।
Hindi: ढाका बांग्लादेश की राजधानी है.</s>

Bengali: রাস্তার দিকে মনোযোগ দিন।
Hindi: सड़क पर ध्यान दें.</s>

Bengali: আমার মাথা ব্যথা করছে
Hindi: मुझे सिर दर्द है.</s>

Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।
Hindi:


['<s> Bengali: খবরটা শুনে খুব খারাপ লাগলো।\nHindi: खबर सुनकर बहुत दुख हुआ.</s>\n\nBengali: এটার দাম কত?\nHindi: इसकी कीमत कितनी होती है?</s>\n\nBengali: ঢাকা বাংলাদেশের রাজধানী।\nHindi: ढाका बांग्लादेश की राजधानी है.</s>\n\nBengali: রাস্তার দিকে মনোযোগ দিন।\nHindi: सड़क पर ध्यान दें.</s>\n\nBengali: আমার মাথা ব্যথা করছে\nHindi: मुझे सिर दर्द है.</s>\n\nBengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।\nHindi: प्रशिक्षण अलगोरिडम, परामिति, न्यायता सीमाबद्धता वा अन्यान्य प्रयोग पद्धति एवं वैशिष्ट्य संपर्क की जानकारी। industries\n\nBengali: খবরটা শুনে খুব খারাপ লাগলো।\nHindi: खबर सुनकर बहुत दुख हुआ. industries\n\nBengali: এটার দাম কত?\nHindi: इसकी कीमत कितनी होती है? industries\n\nBengali: ঢাকা বাংলাদেশের রাজ��']

## LoRa Adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0,  # 0 is optimized (unsloth)
    bias="none",  # "none" is optimized (unsloth)
    use_gradient_checkpointing=True,
    random_state=42,
    use_rslora=False,  # rank stabilized LoRA
    loftq_config=None,  # LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Load dataset

In [1]:
# from datasets import load_dataset

# def formatting_prompts_func(examples):
#     # dict of lists to list of dicts
#     examples = [dict(zip(examples, t)) for t in zip(*examples.values())]
#     texts = []
#     for example in examples:
#         text = (
#             apply_prompt(training=True, eos_token=tokenizer.eos_token, **example)
#             # + EOS_TOKEN  # FIXME: remove from apply_prompt and do here explicitly
#         )
#         texts.append(text)
#     return {
#         "text": texts,
#     }
# name = "bn-hi-parallel-data"
# #dataset_path = f"/storage/praha1/home/rahmang/{name}"
# dataset_path = "grarafat/test-bn-hi-parallel-data"
# # dataset_path = "~/datasets/npfl087-demo-small"
# dataset = load_dataset(dataset_path, split="train", token="give the access token from huggingface")
# dataset = dataset.map(
#     formatting_prompts_func,
#     batched=True,
# )

In [ ]:
# Select the first 5 rows
first_five = dataset.select(range(5))

# Convert to pandas DataFrame and display
df = first_five.to_pandas()
print(df)

  source_lang target_lang                                        source_text  \
0     Bengali       Hindi  ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক নতুন প্র...   
1     Bengali       Hindi  মার্কিন যুক্তরাষ্ট্রের প্রিন্সটন বিশ্ববিদ্যালয...   
2     Bengali       Hindi  "এই বিতর্কটি হারিকেন ক্যাটরিনার সময় ত্রাণ ও পু...   
3     Bengali       Hindi  পুনর্নির্মাণের চেষ্টা সম্পর্কে উদার সমালোচনা ও...   
4     Bengali       Hindi  খেলাটি দুর্দান্ত আবহাওয়ার সাথে সকাল ১০:০০ টায...   

                                         target_text  \
0  जर्नल साइंस में गुरूवार को प्रकाशित एक अध्ययन ...   
1  संयुक्त राज्य अमेरिका में प्रिंसटन विश्वविद्या...   
2  तूफ़ान कैटरीना के मद्देनज़र राहत और पुनर्निर्म...   
3  पुनर्निर्माण की कोशिश की स्पष्ट आलोचना वॉशिंगट...   
4  खेल सुबह 10:00 बजे बहुत बढ़िया मौसम के साथ शुर...   

                                                text  
0  Bengali: ইকুয়েডরীয় গ্যালোপেগোস দ্বীপপুঞ্জের এক...  
1  Bengali: মার্কিন যুক্তরাষ্ট্রের প্রিন্সটন বিশ্...  
2  Bengali: "এই ব

In [ ]:
dataset.shape

(503, 5)

## Start training

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=4096,
    dataset_num_proc=2,
    packing=True,  # Unsloth claim: "can make training 5x faster for short sequences."
    args=TrainingArguments(
        per_device_train_batch_size=2,
        num_train_epochs=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
    ),
)

In [ ]:
!pip install wandb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
!wandb.init()

/bin/bash: -c: line 2: syntax error: unexpected end of file


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
# import os
# import wandb

# # Set your Weights and Biases API key as an environment variable
# os.environ["WANDB_API_KEY"] = "give the access api"

# # Log in to Weights and Biases using the API key
# wandb.login()


In [ ]:
# These stats are copied from the unsloth colab example notebook
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

trainer_stats = trainer.train()

# Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")

GPU = NVIDIA RTX A4000. Max memory = 15.724 GB.
4.789 GB of memory reserved.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 8
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.587700
2,1.708000
3,1.526700
4,1.414600
5,1.283700
6,1.228000
7,1.168100
8,1.100200


283.7293 seconds used for training.
4.73 minutes used for training.
Peak reserved memory = 9.367 GB.
Peak reserved memory % of max memory = 59.571 %.


## Save model

In [ ]:
model.save_pretrained("outputs/mistral-ft-qlora")

/storage/praha1/home/rahmang/envs/unsloth/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Inference

In [ ]:
FastLanguageModel.for_inference(model)
sentence = "প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।"
prompt = apply_prompt(source_lang="Bengali", source_text=sentence, target_lang="Hindi")
print(prompt)
inputs = tokenizer(
    prompt,
    return_tensors="pt",
).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
tokenizer.batch_decode(outputs)

Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।
Hindi:


['<s> Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।\nHindi: प्रशिक्षण अल्गोरिदम, परमिति, न्यायता सीमाबद्धता वा अन्यान्य प्रयोग पद्धति और विषयी का जानकारी।\n\n## Overview\n\nThe Algorithmic Fairness and Accountability Toolkit (AFA Toolkit) is a collection of resources for researchers, practitioners, and policymakers to understand and address algorithmic fairness and accountability. The AFA Toolkit is a living document that will be updated as new resources become available.\n\nThe AFA Toolkit is organized into three sections:\n\n- Algorithmic Fairness and Accountability: This section provides an overview of algorithmic fairness and accountability, including definitions, examples, and resources for further reading.\n- Algorithmic Fairness and Accountability in Practice: This section provides practical guidance for implementing algorithmic fairness and accountability in practice, including best practices, case studies, 

In [ ]:
prompt = apply_prompt_n_shot(EXAMPLE_SENTENCES, 5, eos_token=tokenizer.eos_token, source_lang="Bengali", source_text=sentence, target_lang="Hindi")
print(prompt)
inputs = tokenizer(
    prompt,
    return_tensors="pt",
).to("cuda")
outputs = model.generate(**inputs, max_new_tokens=256, use_cache=True)
tokenizer.batch_decode(outputs)

Bengali: খবরটা শুনে খুব খারাপ লাগলো।
Hindi: खबर सुनकर बहुत दुख हुआ.</s>

Bengali: এটার দাম কত?
Hindi: इसकी कीमत कितनी होती है?</s>

Bengali: ঢাকা বাংলাদেশের রাজধানী।
Hindi: ढाका बांग्लादेश की राजधानी है.</s>

Bengali: রাস্তার দিকে মনোযোগ দিন।
Hindi: सड़क पर ध्यान दें.</s>

Bengali: আমার মাথা ব্যথা করছে
Hindi: मुझे सिर दर्द है.</s>

Bengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।
Hindi:


['<s> Bengali: খবরটা শুনে খুব খারাপ লাগলো।\nHindi: खबर सुनकर बहुत दुख हुआ.</s>\n\nBengali: এটার দাম কত?\nHindi: इसकी कीमत कितनी होती है?</s>\n\nBengali: ঢাকা বাংলাদেশের রাজধানী।\nHindi: ढाका बांग्लादेश की राजधानी है.</s>\n\nBengali: রাস্তার দিকে মনোযোগ দিন।\nHindi: सड़क पर ध्यान दें.</s>\n\nBengali: আমার মাথা ব্যথা করছে\nHindi: मुझे सिर दर्द है.</s>\n\nBengali: প্রশিক্ষণ অ্যালগরিদম, পরামিতি, ন্যায্যতা সীমাবদ্ধতা বা অন্যান্য প্রয়োগ পদ্ধতি এবং বৈশিষ্ট্য সম্পর্কে তথ্য।\nHindi: प्रशिक्षण अल्गोरिदम, परमीति, न्यायता सीमाबद्धता या अन्य प्रयोग पद्धति और विशिष्टता के बारे में जानकारी.</s>']